In [ ]:
import pandas as pd
import numpy as np
import json
import datetime
from google.colab import files
import zipfile
import os
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
from matplotlib import pyplot as plt
from geopy.distance import geodesic

In [ ]:
########################### SPLIT BY TIME PERIOD ###############################

#Take a single dataframe of ship data and split it into a list corresponding to a time frame
def timesplit(dataframe):
  series = []
  dataframe = dataframe.sort_values(by='TimeOfFix')
  starttime = dataframe.iloc[0]['TimeOfFix']
  endtime = dataframe.iloc[-1]['TimeOfFix']
  t = pd.date_range(starttime, endtime, freq='H')
  for i in range(len(t) - 1):
    newframe = dataframe[(dataframe['TimeOfFix'] >= t[i]) & (dataframe['TimeOfFix'] < t[i+1])]
    newframe = newframe[~newframe.index.duplicated(keep='last')]
    series.append(newframe)
  return series

In [ ]:
def clustermetric(x, y):
  c1, c2, c3 = 0.5, 0.2, 0.3 # constants
  x_coords = np.column_stack((x[0], x[1]))
  y_coords = np.column_stack((y[0], y[1]))
  latlongdiff = np.sqrt(np.sum((x_coords - y_coords)**2))
  sogdiff = abs(x[2] - y[2])
  headingdiff = 2 - abs(1 - (abs(x[3] - y[3])/180)) # accounts for distance in rotation
  return latlongdiff*headingdiff

In [ ]:
#Code for calculating intra-cluster distance
from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import euclidean_distances
import numpy as np

# Function to calculate average intra-cluster distance for each cluster using DBSCAN
def calculate_intracluster_distance_dbscan(data, eps, min_samples, unique_clusters):
    # Apply DBSCAN clustering algorithm

    # Initialize a list to store intra-cluster distances for each cluster
    intracluster_distances = []
    num_points_in_cluster = []

    # Iterate over each unique cluster label
    for cluster in unique_clusters:
        if cluster != -1:  # Skip noise points labeled as -1
            # Get points that belong to the current cluster
            cluster_points = data[data['Cluster'] == cluster][['Latitude', 'Longitude']]
            num_points_in_cluster.append(len(cluster_points))
            if len(cluster_points) > 1:  # Ensure there are at least 2 points in the cluster
                # Calculate pairwise Euclidean distances between all points in the cluster
                distances = euclidean_distances(cluster_points, cluster_points)
                # Extract the upper triangular part of the distance matrix (excluding the diagonal)
                upper_triangular_sum = np.sum(distances[np.triu_indices(len(distances), k=1)])
                # Calculate the number of elements in the upper triangular part
                num_elements = (len(cluster_points) * (len(cluster_points) - 1)) / 2
                # Calculate the average distance
                intracluster_distance = upper_triangular_sum / num_elements
                # Append the calculated average intra-cluster distance to the list
                #print(intracluster_distance)
                intracluster_distances.append(intracluster_distance)
            else:
                # If there is only one point in the cluster, the intra-cluster distance is 0
                intracluster_distances.append(0)
        else:
            # Handle noise points separately
            intracluster_distances.append(0)
            num_points_in_cluster.append(0)

    # Return the list of average intra-cluster distances for the current data
    return [intracluster_distances, num_points_in_cluster]

In [ ]:
def clusterdata(data_list, eps, min_samples):
  newdata = pd.DataFrame(columns = ['Hour', 'Latitude', 'Longitude', 'SOG', 'Heading', 'Cluster', 'Size', 'IntraClusterDistance'])
  for i in range(len(data_list)):
    #print(i)
    if data_list[i].shape[0] <= 1:
      continue
    model = DBSCAN(eps = eps, min_samples = min_samples, metric = clustermetric).fit(data_list[i][['Latitude', 'Longitude', 'SOG', 'Heading']])
    labels = model.labels_
    # Calculate intra-cluster distances for the current hour's data
    # isolate unique labels
    uniquelabels = []
    for j in range(len(labels)):
      if labels[j] in uniquelabels:
        continue
      else:
        uniquelabels.append(labels[j])
    uniquelabels.sort()
    tempdata = data_list[i].assign(IntraClusterDistance = np.zeros(data_list[i].shape[0]))
    tempdata.insert(tempdata.shape[1] - 1, 'Cluster', labels, True)
    tempdata.insert(tempdata.shape[1] - 1, 'Size', np.zeros(tempdata.shape[0]))

    tempdata.insert(0, 'AvgHead', np.zeros(tempdata.shape[0]))
    tempdata.insert(0, 'AvgLon', np.zeros(tempdata.shape[0]))
    tempdata.insert(0, 'AvgLat', np.zeros(tempdata.shape[0]))


    for label in uniquelabels:
      latsum = 0
      lonsum = 0
      headsum = 0
      labelcount = 0
      for lat in tempdata[tempdata['Cluster'] == label]['Latitude']:
        latsum += lat
        labelcount += 1
      for lon in tempdata[tempdata['Cluster'] == label]['Longitude']:
        lonsum += lon
      for head in tempdata[tempdata['Cluster'] == label]['Heading']:
        headsum += float(head)
      for row in range(tempdata.shape[0]):
        if tempdata.iloc[row]['Cluster'] == label:
          tempdata.iloc[row,0] = latsum / labelcount
          tempdata.iloc[row,1] = lonsum / labelcount
          tempdata.iloc[row,2] = headsum / labelcount

    distances = calculate_intracluster_distance_dbscan(tempdata, eps, min_samples, uniquelabels)

    for k in range(tempdata.shape[0]):
      if uniquelabels[0] == -1:
        tempdata.iloc[k,13] = distances[0][tempdata.iloc[k]['Cluster'] + 1]
        tempdata.iloc[k,12] = distances[1][tempdata.iloc[k]['Cluster'] + 1]
      else:
        tempdata.iloc[k,13] = distances[0][tempdata.iloc[k]['Cluster']]
        tempdata.iloc[k,12] = distances[1][tempdata.iloc[k]['Cluster']]

    tempdata.insert(0, 'Hour',  np.ones(tempdata.shape[0]) * i)
    tempdata = tempdata[['Hour', 'Latitude', 'Longitude', 'SOG', 'Heading', 'Cluster', 'Size', 'IntraClusterDistance', 'AvgLat', 'AvgLon', 'AvgHead']]
    newdata = pd.concat([newdata, tempdata])
  return newdata

In [ ]:
################### MAIN FUNCTION ######################

# csv should be a file path
def cluster(csv, eps, min_samples):
    df = pd.read_csv(csv)
    trange = time_split(df)
    clustered = clusterdata(trange, eps, min_samples)
    return clustered.to_csv("ClusterData.csv", index = False)